In [4]:
from datasets import *
from datasets.preprocessing import *
import numpy as np

In [20]:
root_path = "E:\\subset"
output_path = "C:\\Users\\jeong\\Desktop\\python\\action-recognition-practice\\preprocessed\\"

In [6]:

def _find_classes(root_path: str) -> Tuple[List[str], Dict[str, int]]:
    """
    `[class명] xxx/yyy/zzz.mp4 or .xml` 폴더 구조에서 class만 추출
    """
    classes = []
    for d in os.scandir(root_path):
        if not d.is_dir():
            continue
        c = d.name[1:3]
        if not c in classes:
            classes.append(c)
    classes.sort()
    class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
    return classes, class_to_idx


def _make_dataset(root_path: str, class_to_idx: Dict[str, int],
                  ext: Optional[Tuple[str, ...]]) -> List[Tuple[str, int]]:
    """
    `[class명] xxx/yyy/zzz.mp4 or .xml` 폴더 구조에서 `list(영상 경로, 클래스)` 변환
    """
    items = []
    directories = [d for d in os.scandir(root_path) if d.is_dir()]
    for d in directories:
        kls = class_to_idx[d.name[1:3]]
        for root, _, filenames in os.walk(d.path, followlinks=True):
            for fn in filenames:
                if fn.lower().endswith(ext):
                    items.append((os.path.join(root, fn), kls))
    return items


In [7]:
classes, class_to_idx = _find_classes(root_path)
samples = _make_dataset(root_path, class_to_idx, ('mp4'))
clips = []
targets = []
num_samples = len(samples)
cnt = 0
starttime = time()
for file, target in samples:
    print(file)
    targets.append(target)
    clips.append(create_clip(file))
    cnt += 1
    elapsed = time() - starttime
    avg = elapsed / cnt
    print(
        f"{cnt}/{num_samples} complete, {round((cnt / num_samples) * 100,2)}%"
    )
    print(
        f"avg. {round(avg,2)}s, estimation. {round(avg * num_samples / 60,2)}m"
    )

02_robbery01_place03_night_summer.mp4
138/525 complete, 26.29%
avg. 3.68s, estimation. 32.2m
E:\subset\[강도] 실외_04\182-3\182-3_cam01_robbery01_place03_night_spring.mp4
139/525 complete, 26.48%
avg. 3.66s, estimation. 32.02m
E:\subset\[강도] 실외_04\182-3\182-3_cam01_robbery01_place03_night_summer.mp4
140/525 complete, 26.67%
avg. 3.64s, estimation. 31.83m
E:\subset\[강도] 실외_04\182-3\182-3_cam02_robbery01_place03_night_spring.mp4
141/525 complete, 26.86%
avg. 3.62s, estimation. 31.64m
E:\subset\[강도] 실외_04\182-3\182-3_cam02_robbery01_place03_night_summer.mp4
142/525 complete, 27.05%
avg. 3.59s, estimation. 31.46m
E:\subset\[납치] 실외_01\199-1\199-1_cam01_kidnap01_place01_day_spring.mp4
143/525 complete, 27.24%
avg. 3.58s, estimation. 31.32m
E:\subset\[납치] 실외_01\199-1\199-1_cam01_kidnap01_place01_day_summer.mp4
144/525 complete, 27.43%
avg. 3.56s, estimation. 31.17m
E:\subset\[납치] 실외_01\199-1\199-1_cam02_kidnap01_place01_day_spring.mp4
145/525 complete, 27.62%
avg. 3.55s, estimation. 31.05m
E:\sub

In [16]:
for i in range(len(clips)):
    np.savez_compressed(output_path+str(i), clip= clips[i], target=targets[i])

In [22]:
np.savez_compressed(output_path +'cfg', classes=classes, lookup=class_to_idx)

In [26]:
np.load(output_path+'cfg.npz', allow_pickle=True)['lookup']

array({'강도': 0, '납치': 1, '침입': 2}, dtype=object)

In [30]:
np.load(output_path+'200.npz', allow_pickle=True)['target']

array(1)